# Trending FourSquare Venues in COVID-19 Hot Spot Zip Codes in Virginia, USA

H. Diana McSpadden

2020-12-29

## Introduction
According to the Virginia Department of Health COVID-19 is spread through respiratory droplets, and is more likely spread when people are in close contact. "Community spread" is the public health term used to describe someone being infected by a disease when they do not know when they were in contact with the sick person. Community spread may be occuring Virginia communities at indoor public venues such as restaurants, bars, coffee shops, and grocery stores. 

Determining what categories of venues are correlated with high per capita COVID-19 infection rates can be helpful to determine community risk factors. This project uses FourSquare venues by Virginia zip codes to determine if such a correlation exists. Machine Learning, specific the Support Vector Machine method, is used to determine if high COVID-19 per capita infections can be predicted based on types of trending venues by zip code.


## The Data

Four data sources are utilized in this project:

1. **World Population Review** provides a csv of Virginia Zip Codes and estimates of the 2020 population for each zip code.
  * These data are required to acquire a dataset of Virginia zip codes, and to determine the per capita COVID-19 infections per zip code.
  * https://worldpopulationreview.com/zips/virginia
2. **Open Data Standard API** will be used to aquire a latitude and longitude for each zip code.
  * **It should be noted** that the FourSquare API can use zip codes with the ***&near*** querystring parameter, but in order for more practice I will retrieve latitudes and longitudes.
  * https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=VA&rows=1275&facet=state&facet=timezone&facet=dst&refine.state=VA
3. **Virginia's Open Data Portal of COVID-19 Counts by Zip Code** provides cummulative COVID-19 counts per Virginia zip code.
  * These data are required to calculate the per capita COVID-19 infections per zip code.
  * Data are included for 5/15/2020 - 12-27-2020: I will use **December 1, 2020 - December 27, 2020** in order to focus on the most current infections, and also to best match venue data which is from December 28, 2020.
  * I subtract total cases for each zip code on November 30th from the total cases on December 27th.
  * I divide the total December cases by zip code population to determine the per capita COVID case count.
  * I then determine the mean per capita case count and assign a categorical label: "low" or "high" to the zip code based on whether the zip code's per capita COVID-19 December cases are above or below the mean.
  * https://data.virginia.gov/Government/VDH-COVID-19-PublicUseDataset-ZIPCode/8bkr-zfqv

4. **FourSquare Venues Endpoint** will provide venues within 2000m (2 km) of the zip code.
  * The FourSquare API will return categories for each venue. **The top 10 categories for each zip code will be calculated, and the resulting data will be used in the Support Vector Machine to determine what accuracy we can predict high or low per capita COVID-19 infections by venue categories.**
  * **It should be noted** that many venues will be beyond the 2000m radius of the FourSquare search. This project is for example purposes only.
  * https://developer.foursquare.com/docs/places-api/endpoints/
  

### Collect the Data

Import needed libraries

In [1]:
# import needed libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

#!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


#!pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


#### Collect Population For Each Virginia Zip Code

Load the csv of population by Virginia zip code from World Population Review.

In [2]:
df_ZipPop = pd.read_csv(r'Data/VirginiaPopByZipCode.csv')
print(df_ZipPop.shape)
df_ZipPop.head()

(892, 4)


,zip,city,county,pop
0,22193,Woodbridge,Prince William,82573
1,23464,Virginia Beach,Virginia Beach City,76495
2,22191,Woodbridge,Prince William,70641
3,23322,Chesapeake,Chesapeake City,65603
4,20147,Ashburn,Loudoun,64197


Need to change data type of the zip column for later merging.

In [3]:
# need to change data type for later merging
df_ZipPop['zip'] = df_ZipPop['zip'].astype(str)
print(df_ZipPop.dtypes)

zip       object
city      object
county    object
pop        int64
dtype: object


#### Collect Latitude and Longitude for each zip code

From Open Data Standard retrieve all Virginia zip codes and corresponding latitudes and longitudes. Save the results in DataFrame and merge the DataFrame with the population DataFrame.

In [4]:
#https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=VA&rows=1275&facet=state&facet=timezone&facet=dst&refine.state=VA

zip_url = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=VA&rows=1275&facet=state&facet=timezone&facet=dst&refine.state=VA'
            
# make the GET request
zip_results = requests.get(zip_url).json()['records']
#print(zip_results)
zip_latlon_list=[]
zip_latlon_list.append([(z['fields']['zip'], z['fields']['latitude'], z['fields']['longitude']) for z in zip_results])

df_ZipLatLon = pd.DataFrame([item for zip_latlon_list in zip_latlon_list for item in zip_latlon_list])
df_ZipLatLon.columns = ['zip', 'Latitude', 'Longitude']

In [5]:
print(df_ZipLatLon.shape)
df_ZipLatLon['zip'] = df_ZipLatLon['zip'].astype(str)
print(df_ZipLatLon.dtypes)
df_ZipLatLon.head()

(1275, 3)
zip           object
Latitude     float64
Longitude    float64
dtype: object


,zip,Latitude,Longitude
0,23887,36.563755,-77.82652
1,24656,37.198005,-82.12193
2,24149,37.011934,-80.41897
3,23125,37.342721,-76.27989
4,22652,38.823987,-78.45310


* Merge the DataFrames
* Drop NA values from the merged DataFrame

In [6]:
df_ZipUnion = df_ZipLatLon.merge(df_ZipPop, on='zip', how='left')

In [7]:
df_ZipUnion.dropna(inplace=True)
print(df_ZipUnion.shape)
df_ZipUnion.head()

(891, 6)


,zip,Latitude,Longitude,city,county,pop
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0
3,23125,37.342721,-76.27989,New Point,Mathews,255.0
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0


Create a new **areaName** column in the DataFrame with CITY, COUNTY (ZIPCODE) for display purposes.

In [8]:
df_ZipUnion['areaName'] = df_ZipUnion['city'] + ', ' + df_ZipUnion['county'] + " (" + df_ZipUnion['zip'] + ')'
df_ZipUnion.head()

,zip,Latitude,Longitude,city,county,pop,areaName
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0,"Valentines, Brunswick (23887)"
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0,"Vansant, Buchanan (24656)"
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0,"Riner, Montgomery (24149)"
3,23125,37.342721,-76.27989,New Point,Mathews,255.0,"New Point, Mathews (23125)"
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0,"Fort Valley, Shenandoah (22652)"


#### Collect Virginia COVID-19 Infection Counts By Zip Code

Load the csv of Virginia's Open Data Portal of COVID-19 Counts by Zip Code 

In [9]:
df_ZipCOVID = pd.read_csv(r'Data/VDH-COVID-19-PublicUseDataset-ZIPCode.csv')
print(df_ZipCOVID.shape)
df_ZipCOVID.head()

(199668, 5)


,Report Date,ZIP Code,Number of Cases,Number of Testing Encounters,Number of PCR Testing Encounters
0,10/29/2020,20105,327,NaN,6316
1,10/29/2020,20106,78,NaN,792
2,10/29/2020,20109,1868,NaN,12598
3,10/29/2020,20110,2327,NaN,15054
4,10/29/2020,20111,1566,NaN,10305


In [10]:
df_ZipCOVID.dtypes

Report Date                          object
ZIP Code                             object
Number of Cases                      object
Number of Testing Encounters        float64
Number of PCR Testing Encounters     object
dtype: object

Set Report Date to datetime format.

In [11]:
df_ZipCOVID['Report Date'] =  pd.to_datetime(df_ZipCOVID['Report Date'], infer_datetime_format=True)
df_ZipCOVID.dtypes

Report Date                         datetime64[ns]
ZIP Code                                    object
Number of Cases                             object
Number of Testing Encounters               float64
Number of PCR Testing Encounters            object
dtype: object

* I only need the Report Date, Zip Code, and Number of Cases columns.
* Drop NaN records
* rename columns to ReportDate, zip, and NumCases
* I only need the values for dates greater than November 30th.

In [12]:
df_ZipCOVID.drop(['Number of Testing Encounters','Number of PCR Testing Encounters'], axis=1, inplace=True)
df_ZipCOVID.dropna(inplace=True)
df_ZipCOVID = df_ZipCOVID[df_ZipCOVID['Number of Cases'] != 'Suppressed']
df_ZipCOVID = df_ZipCOVID[df_ZipCOVID['Number of Cases'] != 'Suppressed*']
print(df_ZipCOVID.shape)


(169113, 3)


In [13]:
# set Num of Cases to int
df_ZipCOVID['Number of Cases'] =  pd.to_numeric(df_ZipCOVID['Number of Cases'])
df_ZipCOVID.dtypes

Report Date        datetime64[ns]
ZIP Code                   object
Number of Cases             int64
dtype: object

In [14]:
# renaming the columns 
df_ZipCOVID.rename({"Report Date": "ReportDate",  
           "ZIP Code": "zip",  
           "Number of Cases": "NumCases"},  
          axis = "columns", inplace = True) 
print(df_ZipCOVID.shape)
df_ZipCOVID.head()

(169113, 3)


,ReportDate,zip,NumCases
0,2020-10-29,20105,327
1,2020-10-29,20106,78
2,2020-10-29,20109,1868
3,2020-10-29,20110,2327
4,2020-10-29,20111,1566


In [15]:
# remove rows with dates less than 2020-11-30
df_ZipCOVIDDecember = df_ZipCOVID[df_ZipCOVID.ReportDate > '2020-11-30']
print(df_ZipCOVIDDecember.shape)
df_ZipCOVIDDecember.head()

(22760, 3)


,ReportDate,zip,NumCases
31712,2020-12-01,20105,455
31713,2020-12-01,20106,106
31714,2020-12-01,20109,2266
31715,2020-12-01,20110,2703
31716,2020-12-01,20111,1922


In [16]:
# groupby zip code, subtract the minimum NumCases from the maximum NumCases to get the December Number Cases
df_ZIPDecCaseCount = df_ZipCOVIDDecember.groupby('zip').agg({'NumCases': ['min', 'max']}) 
print(df_ZIPDecCaseCount.shape)
df_ZIPDecCaseCount.head()

(867, 2)


NumCases      
           min   max
zip                 
20105      455   643
20106      106   140
20109     2266  2786
20110     2703  3367
20111     1922  2487

In [17]:
#rename columns 
df_ZIPDecCaseCount.columns = ['min', 'max']
# calculate the December case counts
df_ZIPDecCaseCount['DecNumCases'] = df_ZIPDecCaseCount['max'] - df_ZIPDecCaseCount['min']

# drop the min and max columns
df_ZIPDecCaseCount.drop(['min','max'], axis=1, inplace=True)
print(df_ZIPDecCaseCount.shape)
df_ZIPDecCaseCount.head()

(867, 1)


,DecNumCases
zip,
20105,188
20106,34
20109,520
20110,664
20111,565


In [18]:
# reset the index
df_ZIPDecCaseCount.reset_index(inplace=True)
print(df_ZIPDecCaseCount.shape)
df_ZIPDecCaseCount.head()

(867, 2)


,zip,DecNumCases
0,20105,188
1,20106,34
2,20109,520
3,20110,664
4,20111,565


**Merge the Num Cases DataFrame with the df_ZipUnion DataFrame**

In [19]:
print("df_ZipUnion Shape: ",df_ZipUnion.shape)
print("df_ZIPDecCaseCount Shape: ",df_ZIPDecCaseCount.shape)

df_ZipUnion Shape:  (891, 7)
df_ZIPDecCaseCount Shape:  (867, 2)


There are some zip codes with 0 cases. Need to make sure the merge works to not eliminate those zips, and I will need to add a 0 value to the NaNs.

In [20]:
df_ZipCOVIDUnion = df_ZipUnion.merge(df_ZIPDecCaseCount, on='zip', how='left')
df_ZipCOVIDUnion.shape

(891, 8)

In [21]:
df_ZipCOVIDUnion.head(10)

,zip,Latitude,Longitude,city,county,pop,areaName,DecNumCases
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0,"Valentines, Brunswick (23887)",0.0
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0,"Vansant, Buchanan (24656)",46.0
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0,"Riner, Montgomery (24149)",52.0
3,23125,37.342721,-76.27989,New Point,Mathews,255.0,"New Point, Mathews (23125)",1.0
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0,"Fort Valley, Shenandoah (22652)",7.0
5,22311,38.837312,-77.12064,Alexandria,Alexandria City,18952.0,"Alexandria, Alexandria City (22311)",260.0
6,24174,37.357587,-79.66552,Thaxton,Bedford,2762.0,"Thaxton, Bedford (24174)",18.0
7,23601,37.053346,-76.45948,Newport News,Newport News City,25578.0,"Newport News, Newport News City (23601)",255.0
8,22578,37.644240,-76.36034,White Stone,Lancaster,2315.0,"White Stone, Lancaster (22578)",23.0
9,23389,37.661513,-75.83150,Harborton,Accomack,258.0,"Harborton, Accomack (23389)",NaN


The tenth row has a NaN, need to set this, and all other NaN's to 0.

In [22]:
df_ZipCOVIDUnion.fillna(0, inplace=True)
df_ZipCOVIDUnion.head(10)

,zip,Latitude,Longitude,city,county,pop,areaName,DecNumCases
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0,"Valentines, Brunswick (23887)",0.0
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0,"Vansant, Buchanan (24656)",46.0
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0,"Riner, Montgomery (24149)",52.0
3,23125,37.342721,-76.27989,New Point,Mathews,255.0,"New Point, Mathews (23125)",1.0
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0,"Fort Valley, Shenandoah (22652)",7.0
5,22311,38.837312,-77.12064,Alexandria,Alexandria City,18952.0,"Alexandria, Alexandria City (22311)",260.0
6,24174,37.357587,-79.66552,Thaxton,Bedford,2762.0,"Thaxton, Bedford (24174)",18.0
7,23601,37.053346,-76.45948,Newport News,Newport News City,25578.0,"Newport News, Newport News City (23601)",255.0
8,22578,37.644240,-76.36034,White Stone,Lancaster,2315.0,"White Stone, Lancaster (22578)",23.0
9,23389,37.661513,-75.83150,Harborton,Accomack,258.0,"Harborton, Accomack (23389)",0.0


### Calculate the per population number of December COVID Cases for each Virginia zip code

In [23]:
df_ZipCOVIDUnion['DecCOVIDPerCap'] = df_ZipCOVIDUnion['DecNumCases'] / df_ZipCOVIDUnion['pop']

In [24]:
df_ZipCOVIDUnion.replace([np.inf, -np.inf], np.nan)
df_ZipCOVIDUnion.fillna(0, inplace=True)
df_ZipCOVIDUnion.dtypes

zip                object
Latitude          float64
Longitude         float64
city               object
county             object
pop               float64
areaName           object
DecNumCases       float64
DecCOVIDPerCap    float64
dtype: object

In [25]:
df_ZipCOVIDUnion.describe()


,Latitude,Longitude,pop,DecNumCases,DecCOVIDPerCap
count,891.000000,891.000000,891.000000,891.000000,891.000000
mean,37.655585,-78.352604,9307.725028,101.529742,inf
std,0.741489,1.770358,13765.797134,153.625160,NaN
min,36.554164,-83.483220,0.000000,0.000000,0.000000
25%,37.023567,-79.360430,831.000000,7.000000,0.006452
50%,37.528701,-77.998750,2777.000000,31.000000,0.009658
75%,38.234870,-77.100440,11063.500000,134.000000,0.013966
max,39.345906,-75.367800,82573.000000,1232.000000,inf


In [26]:
# calculate the thredhold for above average December Cases Per Capita, vs below average December Cases Per Capita
threshold_DecCasesPerCap = df_ZipCOVIDUnion['DecNumCases'].mean() / df_ZipCOVIDUnion['pop'].mean()
threshold_DecCasesPerCap

0.010908115737950074

### Threshold For Above or Below December COVID-19 Cases Per Capita in Virgina Counties: 1.1% of zip code population

Add a column to the df_ZipCOVIDUnion DataFrame with 'above' or 'below' depending on if the DecCOVIDPerCap is > 0.011 or < 0.011

In [27]:
df_ZipCOVIDUnion['aboveorbelow'] = np.where(df_ZipCOVIDUnion['DecCOVIDPerCap'] < 0.011, 'below', 'above')
df_ZipCOVIDUnion.head()

,zip,Latitude,Longitude,city,county,pop,areaName,DecNumCases,DecCOVIDPerCap,aboveorbelow
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0,"Valentines, Brunswick (23887)",0.0,0.000000,below
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0,"Vansant, Buchanan (24656)",46.0,0.016358,above
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0,"Riner, Montgomery (24149)",52.0,0.014731,above
3,23125,37.342721,-76.27989,New Point,Mathews,255.0,"New Point, Mathews (23125)",1.0,0.003922,below
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0,"Fort Valley, Shenandoah (22652)",7.0,0.005228,below


For ongoing working I only need the following columns:
* zip
* Latitude
* Longitude
* pop
* areaName
* aboveorbelow

In [28]:
df_ZipsToUse = df_ZipCOVIDUnion.drop(['city','county','DecNumCases','DecCOVIDPerCap'], axis=1)
print(df_ZipsToUse.shape)
df_ZipsToUse.head()

(891, 6)


,zip,Latitude,Longitude,pop,areaName,aboveorbelow
0,23887,36.563755,-77.82652,387.0,"Valentines, Brunswick (23887)",below
1,24656,37.198005,-82.12193,2812.0,"Vansant, Buchanan (24656)",above
2,24149,37.011934,-80.41897,3530.0,"Riner, Montgomery (24149)",above
3,23125,37.342721,-76.27989,255.0,"New Point, Mathews (23125)",below
4,22652,38.823987,-78.45310,1339.0,"Fort Valley, Shenandoah (22652)",below


#### Collect the venues near the latitude, longitude of the zip codes
For each zip code query the FourSquare API for a max of 50 venues within 2000m (2km) of the zip code lat, lon

In [29]:
CLIENT_ID = 'FHLIENCAGLQPDQRTXZFUBNVHPMW4TRXLFQN0OLHZ3SJXQGQF' 
CLIENT_SECRET = 'VT2WIU3QJHHQUFY04FC4BKHZJIMUA1TNZE4TAKYYUX4QGAEJ' 
VERSION = '20180605'
LIMIT = 100 # A default Foursquare API limit value

In [30]:
names = df_ZipsToUse['areaName']
latitudes = df_ZipsToUse['Latitude']
longitudes = df_ZipsToUse['Longitude']
radius = 2000

In [31]:
explore_venues_list = []

for name, lat, lng in zip(names, latitudes, longitudes):
     
    # create the API request URL
    explore_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

    try:
        # make the GET request
        explore_results = requests.get(explore_url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        explore_venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in explore_results])
    except:
        explore_venues_list.append([('NaN','NaN','NaN','NaN','NaN','NaN','NaN')])

In [32]:
nearby_venues = pd.DataFrame([item for explore_venues_list in explore_venues_list for item in explore_venues_list])
nearby_venues.columns = ['areaName', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

In [33]:
print(nearby_venues.shape)
nearby_venues.head()

(15250, 7)


,areaName,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Riner, Montgomery (24149)",37.011934,-80.41897,bread basket,37.018349,-80.428886,Bakery
1,"Riner, Montgomery (24149)",37.011934,-80.41897,B & B Quality Fencing Inc,37.023586,-80.426829,Moving Target
2,"New Point, Mathews (23125)",37.342721,-76.27989,New Point RV Resort,37.344380,-76.276341,RV Park
3,"New Point, Mathews (23125)",37.342721,-76.27989,Warung Kopi Fauzan Lampineung,37.353405,-76.285563,Café
4,"New Point, Mathews (23125)",37.342721,-76.27989,Tower Coffee,37.353405,-76.285563,Coffee Shop


## Methodology 

### Exploratory Data Analysis

Before training and testing the Support Vector Machine I would like to explore the two DataFrames I have created:

1. df_ZipsToUse: contains the Virginia zip codes, areaNames, and the categorical variables: aboveorbelow which designates whether the zip has December 2020 covid infections above or below the Virginia per capita mean.
2. nearby_venues: contains venues near the areas and the venue category 

#### Exploring df_ZipsToUse

What is the count of above per capita infections zip codes and below per capita zip codes?

In [34]:
df_ZipsToUse.groupby('aboveorbelow').count()

,zip,Latitude,Longitude,pop,areaName
aboveorbelow,,,,,
above,361,361,361,361,361
below,530,530,530,530,530


**361** zip codes are ***above*** the mean December 2020 COVID infections.

**530** zip codes are ***below*** the mean December 2020 COVID infections.

**Where are the above's and below's in Virginia?**

Red dots are zip codes with above average per capita infections in December.

Blue dots are zip codes with below average per capita infections in December.


In [35]:
map_abovebelow = folium.Map(location=[37.926868, -79.124902], zoom_start=7)

# add markers to map
for lat, lng, label, colorcat in zip(df_ZipsToUse['Latitude'], df_ZipsToUse['Longitude'], df_ZipsToUse['areaName'],df_ZipsToUse['aboveorbelow']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color= 'red' if colorcat == 'above' else 'blue',
        fill=True,
        fill_color = 'lightred' if colorcat == 'above' else 'lightblue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_abovebelow)  

map_abovebelow

#### Exploring nearby_venues

What are the counts of the venues in each category?


In [36]:
nearby_venues.groupby('Venue Category').count()

,areaName,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
ATM,14,14,14,14,14,14
Accessories Store,13,13,13,13,13,13
Adult Boutique,4,4,4,4,4,4
Advertising Agency,3,3,3,3,3,3
Afghan Restaurant,6,6,6,6,6,6
...,...,...,...,...,...,...
Wings Joint,46,46,46,46,46,46
Women's Store,27,27,27,27,27,27
Yoga Studio,54,54,54,54,54,54


Here are the top 25 categories of venues in the state or Virginia:

In [37]:
nearby_venues.groupby('Venue Category').count().sort_values(by='areaName', ascending=False).head(25)

,areaName,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Pizza Place,596,596,596,596,596,596
Fast Food Restaurant,531,531,531,531,531,531
Convenience Store,494,494,494,494,494,494
American Restaurant,491,491,491,491,491,491
Sandwich Place,485,485,485,485,485,485
Coffee Shop,386,386,386,386,386,386
Discount Store,330,330,330,330,330,330
Mexican Restaurant,327,327,327,327,327,327
Park,289,289,289,289,289,289


Here are the top 150 venue categories when the areaName has an **above average** per capita infection rate:

In [64]:
df_topVenuesAbove = nearby_venues[nearby_venues['areaName'].isin(df_ZipsToUse[df_ZipsToUse['aboveorbelow'] == 'above']['areaName'])].groupby('Venue Category').count().sort_values(by='areaName', ascending=False).head(150)
df_topVenuesAbove.reset_index(inplace=True)
df_topVenuesAbove.head(150)

,Venue Category,areaName,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude
0,Fast Food Restaurant,233,233,233,233,233,233
1,Pizza Place,225,225,225,225,225,225
2,American Restaurant,186,186,186,186,186,186
3,Sandwich Place,185,185,185,185,185,185
4,Convenience Store,176,176,176,176,176,176
...,...,...,...,...,...,...,...
145,Vineyard,6,6,6,6,6,6
146,Performing Arts Venue,6,6,6,6,6,6
147,Tea Room,6,6,6,6,6,6
148,Shopping Plaza,6,6,6,6,6,6


Here are the top 150 venue categories when the areaName has an **below average** per capita infection rate:

In [65]:
df_bottomVenuesBelow = nearby_venues[nearby_venues['areaName'].isin(df_ZipsToUse[df_ZipsToUse['aboveorbelow'] == 'below']['areaName'])].groupby('Venue Category').count().sort_values(by='areaName', ascending=False).head(150)
df_bottomVenuesBelow.reset_index(inplace=True)
df_bottomVenuesBelow.head(150)

,Venue Category,areaName,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude
0,Pizza Place,371,371,371,371,371,371
1,Convenience Store,318,318,318,318,318,318
2,American Restaurant,305,305,305,305,305,305
3,Sandwich Place,300,300,300,300,300,300
4,Fast Food Restaurant,298,298,298,298,298,298
...,...,...,...,...,...,...,...
145,Optical Shop,13,13,13,13,13,13
146,Health & Beauty Service,13,13,13,13,13,13
147,Moving Target,13,13,13,13,13,13
148,Market,12,12,12,12,12,12


### Findings after EDA

There is not much difference in the categories that make up the top 50 lists. Although a couple things did stand out:

1. While there are fewer zip codes with above average infections, there are a similar number of **Fast Food** restaurants in both the above and below zip codes.
2. **Golf Courses** are not one of the 50 most frequent venue categories in the above average zip codes, but they are #44 in the below average zip codes. 


I only want to use the 150 most frequent categories represented in the top and bottom infection so, I need to filter by those categories.


In [66]:
print('There are {} unique venue categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 460 unique venue categories.


In [67]:
frames = [df_topVenuesAbove, df_bottomVenuesBelow]

#concatenate dataframes
df_VenueCategoriesToUse = pd.concat(frames, sort=False)

print(df_VenueCategoriesToUse.shape)
df_VenueCategoriesToUse.head()

(300, 7)


,Venue Category,areaName,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude
0,Fast Food Restaurant,233,233,233,233,233,233
1,Pizza Place,225,225,225,225,225,225
2,American Restaurant,186,186,186,186,186,186
3,Sandwich Place,185,185,185,185,185,185
4,Convenience Store,176,176,176,176,176,176


In [68]:
print('There are {} unique venue categories.'.format(len(df_VenueCategoriesToUse['Venue Category'].unique())))

There are 164 unique venue categories.


We have **164** unique categories we will use for analysis.

I will filter the nearby_venues DataFrame to only include venues in those categories.

In [69]:
df_VenuesToUse = nearby_venues[nearby_venues['Venue Category'].isin(df_VenueCategoriesToUse['Venue Category'])]
print(df_VenuesToUse.shape)
df_VenuesToUse.head()

(13921, 7)


,areaName,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Riner, Montgomery (24149)",37.011934,-80.41897,bread basket,37.018349,-80.428886,Bakery
1,"Riner, Montgomery (24149)",37.011934,-80.41897,B & B Quality Fencing Inc,37.023586,-80.426829,Moving Target
3,"New Point, Mathews (23125)",37.342721,-76.27989,Warung Kopi Fauzan Lampineung,37.353405,-76.285563,Café
4,"New Point, Mathews (23125)",37.342721,-76.27989,Tower Coffee,37.353405,-76.285563,Coffee Shop
5,"New Point, Mathews (23125)",37.342721,-76.27989,Davis Creek Marina,37.332221,-76.297358,Harbor / Marina


In [70]:
print('There are {} unique areaNames (zip codes).'.format(len(df_VenuesToUse['areaName'].unique())))

There are 735 unique areaNames (zip codes).


Some Virginia zip codes are not represente in the filtered FourSquare DataFrame. There are a total of 891 zip codes in our COVID DataFrame, but some of those zip codes may be too rural to have FourSquare information available.

I will filter the df_ZipsToUse to only inlcude records in the df_VenuesToUse.

In [72]:
df_ZipsToUse = df_ZipsToUse[df_ZipsToUse['areaName'].isin(df_VenuesToUse['areaName'].unique())]
print(df_ZipsToUse.shape)
df_ZipsToUse.head()

(735, 6)


,zip,Latitude,Longitude,pop,areaName,aboveorbelow
2,24149,37.011934,-80.41897,3530.0,"Riner, Montgomery (24149)",above
3,23125,37.342721,-76.27989,255.0,"New Point, Mathews (23125)",below
4,22652,38.823987,-78.45310,1339.0,"Fort Valley, Shenandoah (22652)",below
5,22311,38.837312,-77.12064,18952.0,"Alexandria, Alexandria City (22311)",above
7,23601,37.053346,-76.45948,25578.0,"Newport News, Newport News City (23601)",below


In [76]:
df_ZipsToUse.groupby('aboveorbelow').count()

,zip,Latitude,Longitude,pop,areaName
aboveorbelow,,,,,
above,301,301,301,301,301
below,434,434,434,434,434


Remap the df_ZipsToUse to determine if we still have adequate counts and distribution for a meaningful analysis of Virginia.

In [74]:
map_abovebelow2 = folium.Map(location=[37.926868, -79.124902], zoom_start=7)

# add markers to map
for lat, lng, label, colorcat in zip(df_ZipsToUse['Latitude'], df_ZipsToUse['Longitude'], df_ZipsToUse['areaName'],df_ZipsToUse['aboveorbelow']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color= 'red' if colorcat == 'above' else 'blue',
        fill=True,
        fill_color = 'lightred' if colorcat == 'above' else 'lightblue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_abovebelow2)  

map_abovebelow2

The map and counts appear to still be valid.

### Prepping the venue data for the Support Vector Machine

**Add areaName** to the df_VenuesToUse DataFrame based on the areaName in df_ZipsToUse

In [77]:
df_ZipsVenuesToUse = df_VenuesToUse.merge(df_ZipsToUse, on='areaName', how='left')

In [79]:
df_ZipsVenuesToUse = df_ZipsVenuesToUse.drop(['Latitude_y','Longitude_y'], axis=1)
print(df_ZipsVenuesToUse.shape)
df_ZipsVenuesToUse.head(10)

(13921, 10)


,areaName,Latitude_x,Longitude_x,Venue,Venue Latitude,Venue Longitude,Venue Category,zip,pop,aboveorbelow
0,"Riner, Montgomery (24149)",37.011934,-80.41897,bread basket,37.018349,-80.428886,Bakery,24149,3530.0,above
1,"Riner, Montgomery (24149)",37.011934,-80.41897,B & B Quality Fencing Inc,37.023586,-80.426829,Moving Target,24149,3530.0,above
2,"New Point, Mathews (23125)",37.342721,-76.27989,Warung Kopi Fauzan Lampineung,37.353405,-76.285563,Café,23125,255.0,below
3,"New Point, Mathews (23125)",37.342721,-76.27989,Tower Coffee,37.353405,-76.285563,Coffee Shop,23125,255.0,below
4,"New Point, Mathews (23125)",37.342721,-76.27989,Davis Creek Marina,37.332221,-76.297358,Harbor / Marina,23125,255.0,below
5,"New Point, Mathews (23125)",37.342721,-76.27989,Weis Choice Marina,37.329491,-76.294502,Beach,23125,255.0,below
6,"Fort Valley, Shenandoah (22652)",38.823987,-78.45310,Shenandoah Meadows,38.815174,-78.455619,Campground,22652,1339.0,below
7,"Alexandria, Alexandria City (22311)",38.837312,-77.12064,Stone Hot Pizza,38.840647,-77.120898,Pizza Place,22311,18952.0,above
8,"Alexandria, Alexandria City (22311)",38.837312,-77.12064,Clyde's at Mark Center,38.833405,-77.122974,American Restaurant,22311,18952.0,above
9,"Alexandria, Alexandria City (22311)",38.837312,-77.12064,District Taco,38.846135,-77.120079,Taco Place,22311,18952.0,above


Create **dummy variables** for the Venue Categories.

In [83]:
# one hot encoding
df_Venues_Onehot = pd.get_dummies(df_ZipsVenuesToUse[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_Venues_Onehot['areaName'] = df_ZipsVenuesToUse['areaName']
df_Venues_Onehot['aboveorbelow'] = df_ZipsVenuesToUse['aboveorbelow']

# move neighborhood column to the first column
#fixed_columns = ['areaName','zip','Latitude_x','Latitude_y',] + list(df_Toronto_Onehot.columns[:-1])
#df_Toronto_Onehot = df_Toronto_Onehot[fixed_columns]

df_Venues_Onehot.head(2)

,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,...,Vietnamese Restaurant,Vineyard,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,areaName,aboveorbelow
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,"Riner, Montgomery (24149)",above
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"Riner, Montgomery (24149)",above


In [97]:
df_Venues_Grouped = df_Venues_Onehot.groupby('areaName').mean().reset_index()
df_Venues_Grouped.sort_values(by='areaName',inplace=True)
df_Venues_Grouped.reset_index(inplace=True)
print(df_Venues_Grouped.shape)
df_Venues_Grouped.head()

(735, 166)


,index,areaName,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,...,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,"Abingdon, Washington (24210)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,"Abingdon, Washington (24211)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,"Accomac, Accomack (23301)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,"Afton, Nelson (22920)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,"Alberta, Brunswick (23821)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
print(df_ZipsToUse.shape)
df_ZipsToUse.sort_values(by='areaName',inplace=True)
df_ZipsToUse.reset_index(inplace=True)
df_ZipsToUse.head()

(735, 6)


,index,zip,Latitude,Longitude,pop,areaName,aboveorbelow
0,519,24210,36.728639,-82.00845,16779.0,"Abingdon, Washington (24210)",above
1,659,24211,36.657812,-81.96207,9344.0,"Abingdon, Washington (24211)",above
2,204,23301,37.713561,-75.67108,1580.0,"Accomac, Accomack (23301)",above
3,725,22920,37.987812,-78.79701,3612.0,"Afton, Nelson (22920)",above
4,828,23821,36.872290,-77.91125,1914.0,"Alberta, Brunswick (23821)",below


Add back in the aboveorbelow column to the df_Venues_Grouped

In [98]:
df_Venues_Grouped['aboveorbelow'] = df_ZipsToUse['aboveorbelow']

In [100]:
df_Venues_Grouped.head(10)

,index,areaName,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,...,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,aboveorbelow
0,0,"Abingdon, Washington (24210)",0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,above
1,1,"Abingdon, Washington (24211)",0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,above
2,2,"Accomac, Accomack (23301)",0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,above
3,3,"Afton, Nelson (22920)",0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,above
4,4,"Alberta, Brunswick (23821)",0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,below
5,5,"Aldie, Loudoun (20105)",0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.142857,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,below
6,6,"Alexandria, Alexandria City (22301)",0.044444,0.0,0.0,0.011111,0.000000,0.0,0.000000,0.022222,...,0.000000,0.000000,0.0,0.000000,0.0,0.022222,0.011111,0.0,0.011111,below
7,7,"Alexandria, Alexandria City (22302)",0.058824,0.0,0.0,0.029412,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,above
8,8,"Alexandria, Alexandria City (22304)",0.010989,0.0,0.0,0.000000,0.010989,0.0,0.010989,0.000000,...,0.010989,0.010989,0.0,0.010989,0.0,0.000000,0.010989,0.0,0.021978,above
9,9,"Alexandria, Alexandria City (22305)",0.000000,0.0,0.0,0.010638,0.000000,0.0,0.000000,0.010638,...,0.000000,0.000000,0.0,0.000000,0.0,0.010638,0.010638,0.0,0.010638,below


## Results

** TO DO**
Discuss the results.

## Discussion

1. Had to use explore instead of trending
2. Data will be skewed to places that have FourSquare data available.
3. Used the 150 most frequent Venue Categories in both the above and below segments. If I had used all 406 Venue Categories we may have seen something different, but the calculation time would have increase dramatically.
4. People travel outside their zip code, so venue categories should not be thought of as predicting where people actually have acquired COVID-19 by community spread.

## Conclusion

** TO DO**
Conclude the report.

## References
 Virgina Departments of Health: https://www.vdh.virgi
 nia.gov/coronavirus/frequently-asked-questions/general-questions/